In [1]:
import psycopg2
import pandas as pd
import numpy as np
import mariadb
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance
import pickle
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import seaborn as sns

C:\Users\tommy\AppData\Roaming\Python\Python38\site-packages\scipy\__init__.py:173: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
os.getcwd()

'D:\\Cropnuts\\DSML158\\SoilAnalysis'

In [3]:
soil_df = pd.read_csv("output/soil_analysis_cleaned.csv")

In [4]:
soil_df

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,7,7,3,3,CF045SA0784,2012-07-09 00:00:00.000,Complete Soil Analysis,ec_salts,61.0,uS/cm
1,58,58,29,29,CF045SA0786,2012-07-09 00:00:00.000,Complete Soil Analysis,ec_salts,240.0,uS/cm
2,90,90,45,45,CF045SA0787,2012-07-09 00:00:00.000,Complete Soil Analysis,ec_salts,30.0,uS/cm
3,122,122,61,61,CF045SA0788,2012-07-09 00:00:00.000,Complete Soil Analysis,ec_salts,119.0,uS/cm
4,177,177,95,95,CF045SA0797,2012-07-09 00:00:00.000,Complete Soil Analysis,ec_salts,39.0,uS/cm
...,...,...,...,...,...,...,...,...,...,...
27801,1104,1104,696,696,CW015SA0046,2012-03-05 00:00:00.000,Heavy Metals Analysis,aluminium,639.0,ppm
27802,1189,1189,713,713,CW015SA0047,2012-03-05 00:00:00.000,Heavy Metals Analysis,aluminium,254.0,ppm
27803,1239,1239,723,723,CW015SA0048,2012-03-05 00:00:00.000,Heavy Metals Analysis,aluminium,923.0,ppm
27804,13717,13717,7126,7126,CW015SA0043,2012-02-29 00:00:00.000,Heavy Metals Analysis,aluminium,1395.0,ppm


In [5]:
new_samples_df = soil_df.loc[soil_df['batch_date'] > '2024-05-21'] 
soil_df = soil_df.loc[soil_df['batch_date'] <= '2024-05-21'] 

In [6]:
soil_df['chemical_name'] = [ i.lower().replace(" ","_").replace(".","").replace("(","").replace(")","").replace("/","").strip() for i in soil_df['chemical_name'] ]

In [7]:
for analysis in np.unique(soil_df['analysis_name']):
    print(analysis)
    df_ = soil_df.loc[soil_df['analysis_name']==analysis]
    df_ = pd.pivot_table(data=df_, values="result", index="sample_code", columns="chemical_name")
    print(len(df_))

    os.makedirs(f"output/boxplots/{analysis}",exist_ok=True)
    for column in df_.columns:
        # print(column)
        os.makedirs(f"output/chemical_null_count",exist_ok=True)
        plt.boxplot(df_[column])
        plt.savefig(f"output/boxplots/{analysis}/{column}.png")
        plt.clf()
        outlier_threshold = df_[column].quantile(0.99)
        # df_ = df_.loc[df_[column] <= outlier_threshold]
    df_.describe().to_csv(f"output/chemical_null_count/{analysis}.csv")
    print(len(df_))
    if(len(df_) == 0):
        continue
    os.makedirs(f"output/analysis",exist_ok=True)
    df_.to_csv(f"output/analysis/{analysis}.csv")
    

% Organic Matter (OM)
529
529
% Soil Nitrogen (N)
299
299
AFSIS Std Wet Chemistry Soil Analysis
185
185
Basic Soil Analysis (BSA)
23
23
Carbon Analysis
54
54
Complete Soil Analysis
866
866
Complete Soil Analysis (M3)
6
6
Complete Soil Analysis with Recommendations
65
65
Exch. Aluminium
66
66
Exchangeable Acidity (Hp) Analysis
28
28
Heavy Metals Analysis
6
6
Olsen P Analysis
55
55
SOIL_Scan
15
15
Soil Bulk Density Analysis
22
22
Soil Health Care Analysis
69
69
Soil Texture Analysis
63
63
Standard Soil (Olsen P)
243
243


<Figure size 640x480 with 0 Axes>

In [8]:
# for analysis_file in os.listdir("output/analysis"):
#     os.makedirs("./output/pairplots",exist_ok=True)
#     analysis_df = pd.read_csv(f"output/analysis/{analysis_file}",index_col=0)
#     print(analysis_file.replace('.csv',''))
#     sns.pairplot(analysis_df)
#     print('Saving')
#     plt.savefig(f"output/pairplots/{analysis_file.replace('.csv','')}.png")
#     print('Saved')
#     plt.clf()

In [9]:
for analysis_file in os.listdir("output/analysis"):
    print(analysis_file)
    analysis_df = pd.read_csv(f"output/analysis/{analysis_file}",index_col=0)
    print(len(analysis_df.columns))
    os.makedirs(f"models/imputers",exist_ok=True)
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    analysis_df = pd.DataFrame(imp_mean.fit_transform(analysis_df), columns=analysis_df.columns, index=analysis_df.index)
    pickle.dump(imp_mean, open(f"models/imputers/{analysis_file.replace('csv','pkl')}","wb"))
    # if(len(analysis_df) == 0):
    #     continue
    sc = StandardScaler()
    pca = PCA()
    analysis_scaled = sc.fit_transform(analysis_df)
    pca.fit(analysis_scaled)
    pca_explained_variance_df = pd.DataFrame(zip(analysis_df.columns,pca.explained_variance_ratio_))
    os.makedirs(f"output/pca_explained_variance_ratio",exist_ok=True)
    pca_explained_variance_df.to_csv(f"output/pca_explained_variance_ratio/{analysis_file}")
    pca_explained_variance_df = pca_explained_variance_df.loc[pca_explained_variance_df[1]>0.1]
    n_components = len(pca_explained_variance_df)
    
    pca_chems = pca_explained_variance_df[0]
    
    pca = PCA(n_components=n_components)
    analysis_scaled = sc.fit_transform(analysis_df)
    pca_reduced_df = pca.fit_transform(analysis_scaled)
    analysis_scaled = pd.DataFrame(analysis_scaled,index=analysis_df.index)
    pca_reduced_df = pd.DataFrame(pca_reduced_df,index=analysis_df.index, columns=pca_chems)
    os.makedirs(f"output/analysis_scaled",exist_ok=True)
    os.makedirs(f"output/pca_df",exist_ok=True)
    os.makedirs(f"models/scalers",exist_ok=True)
    os.makedirs(f"models/pca",exist_ok=True)
    analysis_scaled.to_csv(f"output/analysis_scaled/{analysis_file}")
    pca_reduced_df.to_csv(f"output/pca_df/{analysis_file}")
    pickle.dump(sc, open(f"models/scalers/{analysis_file.replace('csv','pkl')}","wb"))
    pickle.dump(pca, open(f"models/pca/{analysis_file.replace('csv','pkl')}","wb"))

% Organic Matter (OM).csv
18
% Soil Nitrogen (N).csv
18
AFSIS Std Wet Chemistry Soil Analysis.csv
1
Basic Soil Analysis (BSA).csv
1
Carbon Analysis.csv
2
Complete Soil Analysis (M3).csv
1
Complete Soil Analysis with Recommendations.csv
3
Complete Soil Analysis.csv
18
Exch. Aluminium.csv
1
Exchangeable Acidity (Hp) Analysis.csv
2
Heavy Metals Analysis.csv
16
Olsen P Analysis.csv
16
Soil Bulk Density Analysis.csv
2
Soil Health Care Analysis.csv
1
Soil Texture Analysis.csv
1
SOIL_Scan.csv
1
Standard Soil (Olsen P).csv
1


In [10]:
for analysis_file in os.listdir("output/pca_df"):
    print(analysis_file)
    os.makedirs(f"output/mahalanobis_distance",exist_ok=True)
    pca_reduced_df = pd.read_csv(f"output/pca_df/{analysis_file}",index_col=0)
    # if len(pca_reduced_df.columns) < 2:
        # continue
    mu = np.mean(pca_reduced_df, axis=0)
    sigma = np.cov(pca_reduced_df.T)
    os.makedirs(f"output/pca_df",exist_ok=True)
    try:
        pca_reduced_df['mahalanobis_distance'] = [distance.mahalanobis(pca_reduced_df.iloc[i], mu, np.linalg.inv(sigma)) for i in range(len(pca_reduced_df)) ]
    except:
        continue
    pca_reduced_df.to_csv(f"output/mahalanobis_distance/{analysis_file}")

% Organic Matter (OM).csv
% Soil Nitrogen (N).csv
AFSIS Std Wet Chemistry Soil Analysis.csv
Basic Soil Analysis (BSA).csv
Carbon Analysis.csv
Complete Soil Analysis (M3).csv
Complete Soil Analysis with Recommendations.csv
Complete Soil Analysis.csv
Exch. Aluminium.csv
Exchangeable Acidity (Hp) Analysis.csv
Heavy Metals Analysis.csv
Olsen P Analysis.csv
Soil Bulk Density Analysis.csv
Soil Health Care Analysis.csv
Soil Texture Analysis.csv
SOIL_Scan.csv
Standard Soil (Olsen P).csv


In [11]:
mahalanobis_threshold_dict = {}
for analysis_file in os.listdir("output/mahalanobis_distance"):
    print(analysis_file)
    os.makedirs(f"output/mahalanobis_distance_upper_quantile",exist_ok=True)
    os.makedirs(f"output/mahalanobis_boxplots",exist_ok=True)
    analysis = analysis_file.replace(".csv","")
    mahalanobis_df = pd.read_csv(f"output/mahalanobis_distance/{analysis_file}",index_col=0)
    upper_quantile = (mahalanobis_df['mahalanobis_distance'].quantile(0.95))
    mahalanobis_threshold_dict[analysis_file.replace(".csv","")] = upper_quantile
    mahalanobis_df.loc[mahalanobis_df['mahalanobis_distance'] >= upper_quantile].to_csv(f"output/mahalanobis_distance_upper_quantile/{analysis_file}")
    plt.boxplot(mahalanobis_df['mahalanobis_distance'])
    plt.savefig(f"output/mahalanobis_boxplots/{analysis}.png")
    plt.clf()
pickle.dump(mahalanobis_threshold_dict, open("mahalanobis_thresholds.dict","wb"))

% Organic Matter (OM).csv
% Soil Nitrogen (N).csv
Carbon Analysis.csv
Complete Soil Analysis with Recommendations.csv
Complete Soil Analysis.csv
Exchangeable Acidity (Hp) Analysis.csv
Heavy Metals Analysis.csv
Olsen P Analysis.csv
Soil Bulk Density Analysis.csv


<Figure size 640x480 with 0 Axes>

In [12]:
mahalanobis_threshold_dict

{'% Organic Matter (OM)': 2.3648505715180743,
 '% Soil Nitrogen (N)': 2.341762560170896,
 'Carbon Analysis': 1.9769038128646055,
 'Complete Soil Analysis with Recommendations': 3.5182055578406537,
 'Complete Soil Analysis': 2.609495967521605,
 'Exchangeable Acidity (Hp) Analysis': 2.8418876183133563,
 'Heavy Metals Analysis': 2.041120041840065,
 'Olsen P Analysis': 5.1603069382258315,
 'Soil Bulk Density Analysis': 1.6971815684432907}

In [13]:
conn_lims = pyodbc.connect("Driver={SQL Server};"
                            "Server=192.168.5.18\CROPNUT;"
                            "Database=cropnuts;"
                            "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

In [14]:
analysis_dict = pd.read_sql(f"""SELECT analysis_id, LTRIM(RTRIM(analysis_name)) as analysis_name FROM Analysis  ORDER BY analysis_name""",con=conn_lims).set_index("analysis_id").to_dict()['analysis_name']
pickle.dump(analysis_dict, open("analysis.dict","wb"))

C:\Users\tommy\AppData\Roaming\Python\Python38\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [15]:
reverse_analysis_dict = pd.read_sql(f"""SELECT analysis_id, LTRIM(RTRIM(analysis_name)) as analysis_name FROM Analysis ORDER BY analysis_name""",con=conn_lims).set_index("analysis_name").to_dict()['analysis_id']
reverse_analysis_dict

C:\Users\tommy\AppData\Roaming\Python\Python38\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


{'% Gypsum': 540,
 '% Nitrogen (RSSP 2)': 647,
 '% Organic Matter': 258,
 '% Organic Matter (RSSP 2)': 646,
 '% Soil Nitrogen': 18,
 '%Assay': 27235,
 '%N (RSSP 2)': 652,
 '%OM (RSSP 2)': 651,
 '%P,%S Analysis(Super Calcium)': 383,
 '1:2 Soil Volume Extract': 294,
 '1:2 soil volume extract': 25,
 '1:2 Soil Volume Extract (Data Only)': 27339,
 '1:2 Vol Extract for BLGG': 317,
 '20:12:12 (N,P205,K20) Analysis': 840,
 'Absorbance at 254nm': 26936,
 'Acid Detergent Fibre': 27063,
 'Acid Insoluble Ash': 27417,
 'Acid Insoluble Matter': 27108,
 'Acid titration': 26647,
 'Acid Value': 26971,
 'Advanced Biological Farming Soil Audit': 26521,
 'Advanced Soil Health Analysis': 27181,
 'Aerobic Mesophilic Count': 26649,
 'Aflatoxin': 26794,
 'Aflatoxin AFB1': 26938,
 'Aflatoxin B1': 26939,
 'Aflatoxin in feed': 767,
 'Aflatoxin Total': 26753,
 'AfSIS Standard Leaf Analysis': 633,
 'AFSIS Std Wet Chemistry Soil Analysis': 589,
 'Aggregate Stability': 26727,
 'Agrifi Soil Microbiome Analysis': 2729

In [20]:
test = []
for sample in soil_df.sample_code.unique():
    res = {}
    tmp_ = soil_df.loc[soil_df.sample_code == sample]
    res['sample_code'] = sample
    res['analysis_id'] = [ reverse_analysis_dict[i] for i in tmp_.analysis_name.unique() if i in reverse_analysis_dict.keys() ]
    for index,row in tmp_.iterrows():
        if row['result'] >= 0 :
            res[row['chemical_name']] = {'result': row['result'], 'units': str(row['unit_name'])}
        else:
            res[row['chemical_name']] = {'result': 0, 'units': row['unit_name']}
    test.append(res)    


['Complete Soil Analysis' '% Organic Matter (OM)']
['Complete Soil Analysis' '% Organic Matter (OM)']
['Complete Soil Analysis' '% Organic Matter (OM)']
['Complete Soil Analysis' '% Organic Matter (OM)']
['Complete Soil Analysis' '% Organic Matter (OM)']
['Complete Soil Analysis' '% Organic Matter (OM)']
['Complete Soil Analysis' '% Organic Matter (OM)']
['Complete Soil Analysis' '% Organic Matter (OM)']
['Complete Soil Analysis']
['Complete Soil Analysis']
['Complete Soil Analysis']
['Complete Soil Analysis' '% Organic Matter (OM)' '% Soil Nitrogen (N)']
['Complete Soil Analysis' '% Organic Matter (OM)' '% Soil Nitrogen (N)']
['Complete Soil Analysis']
['Complete Soil Analysis']
['Complete Soil Analysis']
['Complete Soil Analysis']
['Complete Soil Analysis']
['Complete Soil Analysis']
['Complete Soil Analysis' '% Organic Matter (OM)' '% Soil Nitrogen (N)'
 'Olsen P Analysis' 'Heavy Metals Analysis']
['Complete Soil Analysis' 'Exchangeable Acidity (Hp) Analysis']
['Complete Soil Analys

In [23]:
str(test).replace("'",'"')

'[{"sample_code": "CF045SA0784", "analysis_id": [], "ec_salts": {"result": 61.0, "units": "uS/cm"}, "aluminium": {"result": 481.0, "units": "ppm"}, "cec": {"result": 16.6, "units": "meq/100g"}, "iron": {"result": 165.0, "units": "ppm"}}, {"sample_code": "CF045SA0786", "analysis_id": [], "ec_salts": {"result": 240.0, "units": "uS/cm"}, "aluminium": {"result": 680.0, "units": "ppm"}, "boron": {"result": 0.5, "units": "ppm"}, "calcium": {"result": 6170.0, "units": "ppm"}, "zinc": {"result": 2.06, "units": "ppm"}, "cec": {"result": 46.8, "units": "meq/100g"}, "copper": {"result": 12.5, "units": "ppm"}, "sulphur": {"result": 12.0, "units": "ppm"}, "sodium": {"result": 53.0, "units": "ppm"}, "potassium": {"result": 877.0, "units": "ppm"}, "manganese": {"result": 186.0, "units": "ppm"}, "magnesium": {"result": 1380.0, "units": "ppm"}, "iron": {"result": 98.4, "units": "ppm"}}, {"sample_code": "CF045SA0787", "analysis_id": [], "ec_salts": {"result": 30.0, "units": "uS/cm"}, "ph": {"result": 6.

In [27]:
req_body = test[0:2]
_ = pd.DataFrame(req_body)
unit_decision = pd.read_csv("soil_unit_per_chemical_decision.csv")
analysis_dict = pickle.load(open("analysis.dict","rb"))
mahalanobis_thresholds = pickle.load(open("mahalanobis_thresholds.dict","rb"))
result = {}
_df = pd.DataFrame()
for index, row in _.iterrows():
    analyses = row['analysis_id']
    for analysis_ in analyses:
        df_analysis_ = pd.DataFrame(row).T
        df_analysis_['analysis_name'] = analysis_dict[analysis_]
        _df = pd.concat([_df, df_analysis_])
for index,row in _df.iterrows():
    sample_code = row['sample_code']
    analysis = row['analysis_name']
    analysis_id = row['analysis_id']
    if sample_code not in result.keys():
        result[sample_code] = []
    if analysis not in mahalanobis_thresholds.keys():
        result[sample_code].append({"sample_code": sample_code,"status":"warning", "message": f"Analysis not in models", "details": f"Analysis: {analysis} is not in the list of defined models" })   
        continue     
    scaler = pickle.load(open(f"scalers/{analysis}.pkl","rb"))
    pca = pickle.load(open(f"pca/{analysis}.pkl","rb"))

    pca_df = pd.read_csv(f"pca_df/{analysis}.csv",index_col=0)
    analysis_df = pd.read_csv(f"analysis/{analysis}.csv",index_col=0)

    

    try :
        tmp_df = pd.DataFrame(row).T[analysis_df.columns]
        # tmp_df = tmp_df.dropna() 
    except:
        result[sample_code].append({"sample_code": sample_code,"status":"warning", "message": f"Missing parameters for analysis_id: {analysis}", "details": f"Expected parameters are {','.join(pca_df.columns)} for analysis: {analysis}" })
        continue
    failed_units_comparison = {}    
    for col in pca_df.columns:
        expected_units = unit_decision.loc[(unit_decision['crop'] == analysis) & (unit_decision['chemical_name'] == col)]
        print(expected_units[['crop','chemical_name','unit_name']].to_dict())
        import math
        if type(row[col]) != dict and math.isnan(row[col]):
            failed_units_comparison[col] = expected_units[['crop','chemical_name','unit_name']].to_dict()
        elif row[col]['units'] !=   expected_units['unit_name'].values[0] :
            failed_units_comparison[col] = expected_units[['crop','chemical_name','unit_name']].to_dict()
        else:
            row[col] = row[col]['result']
    if len(failed_units_comparison.keys()) > 0:
        result[sample_code].append({"sample_code": sample_code,"status":"warning", "message": f"Wrong units provided", "details": f"Expected units are {str(failed_units_comparison)} for analysis: {analysis}" })
        continue

    tmp_df = pd.DataFrame(row).T[analysis_df.columns]
    print(tmp_df.columns)
    # tmp_df = 
    df_scaled = scaler.transform(tmp_df)
    df_pca = pd.DataFrame(pca.transform(df_scaled))

    mu = np.mean(pca_df, axis=0)
    sigma = np.cov(pca_df.T)

    mahalanobis_distance = distance.mahalanobis(df_pca.iloc[0], mu, np.linalg.inv(sigma))

    print(mahalanobis_distance)

    expected_md = mahalanobis_thresholds[analysis]
    print(expected_md)

    if mahalanobis_distance > expected_md:
        result[sample_code].append({"sample_code": sample_code,"status":"fail", "message": "Mahalanobis distance exceeds threshold", "description":f"Mahalanobis distance of {mahalanobis_distance} exceeds threshold of {expected_md} for analysis: {analysis}" })
    else:
        result[sample_code].append({"sample_code": sample_code,"status":"pass","message": "Mahalanobis distance within threshold", "description":f"Mahalanobis distance of {mahalanobis_distance} is within threshold of {expected_md} for analysis: {analysis}" })
            


   sample_code analysis_id                             ec_salts  \
0  CF045SA0784          []   {'result': 61.0, 'units': 'uS/cm'}   
1  CF045SA0786          []  {'result': 240.0, 'units': 'uS/cm'}   

                           aluminium                                    cec  \
0  {'result': 481.0, 'units': 'ppm'}  {'result': 16.6, 'units': 'meq/100g'}   
1  {'result': 680.0, 'units': 'ppm'}  {'result': 46.8, 'units': 'meq/100g'}   

                                iron                            boron  \
0  {'result': 165.0, 'units': 'ppm'}                              NaN   
1   {'result': 98.4, 'units': 'ppm'}  {'result': 0.5, 'units': 'ppm'}   

                              calcium                              zinc  \
0                                 NaN                               NaN   
1  {'result': 6170.0, 'units': 'ppm'}  {'result': 2.06, 'units': 'ppm'}   

                             copper                           sulphur  \
0                               NaN      